In [5]:
### RAG ### ConversationBufferMemory사용, LCEL로 stuff chain 구현
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader 
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache2/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,  
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs,cached_embeddings) 

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=200,
    memory_key="chat_history"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer question using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{question}"),
    ]
)

def load_memory(_):  #_는 변수명이나 메소드 내부에서는 입력값이 사용되지 않음을 나타내는 관례적 표현
    return memory.load_memory_variables({})["chat_history"]

chain = {
        "context": retriever,
        "question": RunnablePassthrough(),
        "chat_history": RunnableLambda(load_memory),
        } | prompt | llm

def run_chain(inputs):
    result = chain.invoke(inputs)
    print(result.content)
    memory.save_context(
        {"inputs":inputs},
        {"outputs":"".join(result.content)},  #chain을 호출한 후 결과값으로 content가 반환됨!!
        )
    
run_chain("Is Aaronson guilty?")

According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


In [7]:
run_chain("What message did he write in the table?")

The message he wrote on the table was "2+2=5."


In [8]:
run_chain("Who is Julia?")

Julia is a character in the text who is associated with the protagonist and plays a significant role in the story.
